In [1]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.callbacks import LambdaCallback

nltk.download('punkt')

def get_content(title):
    url = f"https://en.wikipedia.org/wiki/{title}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve Wikipedia page for {title}")
        return None

# Function to preprocess text
def preprocess_text(raw_text):
    clean_text = BeautifulSoup(raw_text, 'html.parser').get_text()
    clean_text = re.sub(r'\[.*?\]', '', clean_text)
    clean_text = re.sub(r'https?://\S+|www\.\S+', '', clean_text)
    clean_text = re.sub(r'\{\|[\s\S]*?\|\}', '', clean_text)
    return clean_text if isinstance(clean_text, str) else str(clean_text)

# Get the Wikipedia content for Prince Philip, Duke of Edinburgh
wiki_title = "Prince_Philip,_Duke_of_Edinburgh"
raw_content = get_content(wiki_title)

if raw_content:
    preprocessed_text = preprocess_text(raw_content)

    maxlen = 40
    step = 3
    sentences = []
    next_chars = []

    for i in range(0, len(preprocessed_text) - maxlen, step):
        sentences.append(preprocessed_text[i: i + maxlen])
        next_chars.append(preprocessed_text[i + maxlen])

    chars = sorted(list(set(preprocessed_text)))
    print('Total characters:', len(chars))
    char_indices = dict((char, chars.index(char)) for char in chars)
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # Vectorization
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
    y = np.zeros((len(sentences), len(chars)), dtype=bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

    # Define RNN model
    model = Sequential()
    model.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
    model.add(Dense(len(chars), activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Helper function to sample an index from a probability array
    def sample(preds, temperature=1.0):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    # Function to generate text after each epoch
    def on_epoch_end(epoch, _):
        # print(f"\n----- Generating text after Epoch: {epoch}")

        start_index = random.randint(0, len(preprocessed_text) - maxlen - 1)
        for temperature in [0.2, 0.5, 1.0, 1.2]:

            generated_text = preprocessed_text[start_index: start_index + maxlen]


            for _ in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.


    # Callback to generate text after each epoch
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    # Train the model
    model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

    # Generate text by predicting the next character for at least 10 iterations
    def generate_text(model, seed, char_indices, indices_char, length=400):
        generated_text = seed
        for _ in range(length):
            sampled = np.zeros((1, len(seed), len(char_indices)))
            for t, char in enumerate(seed):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature=1.0)
            next_char = indices_char[next_index]

            generated_text += next_char
            seed = seed[1:] + next_char

        return generated_text

    seed_start = random.randint(0, len(preprocessed_text) - maxlen - 1)
    seed_text = preprocessed_text[seed_start:seed_start + maxlen]
    print(f"Seed text: {seed_text}")
    generated_text = generate_text(model, seed_text, char_indices, indices_char)
    print(f"Generated text: {generated_text}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Total characters: 290
Epoch 1/60
301/301 [==============================] - 14s 36ms/step - loss: 3.4991
Epoch 2/60
301/301 [==============================] - 9s 30ms/step - loss: 2.9056
Epoch 3/60
301/301 [==============================] - 7s 24ms/step - loss: 2.4944
Epoch 4/60
301/301 [==============================] - 9s 29ms/step - loss: 2.2880
Epoch 5/60
301/301 [==============================] - 9s 29ms/step - loss: 2.1629
Epoch 6/60
301/301 [==============================] - 7s 24ms/step - loss: 2.0750
Epoch 7/60
301/301 [==============================] - 9s 29ms/step - loss: 2.0100
Epoch 8/60
301/301 [==============================] - 7s 24ms/step - loss: 1.9534
Epoch 9/60
301/301 [==============================] - 9s 28ms/step - loss: 1.9035
Epoch 10/60
301/301 [==============================] - 8s 28ms/step - loss: 1.8620
Epoch 11/60
301/301 [==============================] - 8s 25ms/step - loss: 1.8226
Epoch 12/60
301/301 [==============================] - 9s 31ms/step - los

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding

# Function to fetch and preprocess text from Wikipedia page
def fetch_and_preprocess_text(url):
    # Fetch content from Wikipedia page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the main content of the page
    content = soup.find(id='mw-content-text').get_text()
    # Remove square brackets and their content (typically citations)
    content = re.sub(r'\[.*?\]', '', content)
    # Remove newlines and extra whitespaces
    content = re.sub(r'\n+', ' ', content)
    content = re.sub(r'\s+', ' ', content)
    return content

# Wikipedia URL of Prince Philip, Duke of Edinburgh
wiki_url = "https://en.wikipedia.org/wiki/Prince_Philip,_Duke_of_Edinburgh"

# Fetch and preprocess text
text = fetch_and_preprocess_text(wiki_url)

# Reduce the amount of training data
text_subset = text[:1000]

# Tokenize text into words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_subset])
total_words = len(tokenizer.word_index) + 1

# Create input sequences using the tokenized words
input_sequences = []
for line in text_subset.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for uniform length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
X, y = input_sequences[:,:-1],input_sequences[:,-1]
y = np.array(y)

# Define and train the RNN model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=250, verbose=1)

def generate_text_word(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


# Generate text
seed_word = "Prince"
generated_text = generate_text_word(seed_word, 10, max_sequence_len)
print(generated_text)


Epoch 1/250
5/5 [==============================] - 2s 68ms/step - loss: 4.6402 - accuracy: 0.0145
Epoch 2/250
5/5 [==============================] - 0s 69ms/step - loss: 4.4501 - accuracy: 0.0942
Epoch 3/250
5/5 [==============================] - 0s 72ms/step - loss: 4.3428 - accuracy: 0.1449
Epoch 4/250
5/5 [==============================] - 0s 68ms/step - loss: 4.2484 - accuracy: 0.2536
Epoch 5/250
5/5 [==============================] - 0s 66ms/step - loss: 4.1515 - accuracy: 0.3551
Epoch 6/250
5/5 [==============================] - 0s 71ms/step - loss: 4.0751 - accuracy: 0.3478
Epoch 7/250
5/5 [==============================] - 0s 73ms/step - loss: 3.9668 - accuracy: 0.5000
Epoch 8/250
5/5 [==============================] - 0s 69ms/step - loss: 3.8795 - accuracy: 0.5290
Epoch 9/250
5/5 [==============================] - 0s 72ms/step - loss: 3.7644 - accuracy: 0.6159
Epoch 10/250
5/5 [==============================] - 0s 57ms/step - loss: 3.6560 - accuracy: 0.7319
Epoch 11/250
5/5 [=